In [1]:
from PIL import Image
import numpy as np
import zipfile
from os import walk
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.model_selection import train_test_split

import keras
from math import ceil
from keras.preprocessing.image import ImageDataGenerator
from keras.applications import ResNet50
from keras.applications.resnet50 import preprocess_input
from keras.callbacks import EarlyStopping
from keras import Model, layers
from keras.models import load_model, model_from_json

Using TensorFlow backend.


## 2. Prepare the Images

In [2]:
train_datagen = ImageDataGenerator(
    shear_range=10,
    zoom_range=0.2,
    horizontal_flip=True,
    preprocessing_function=preprocess_input)

train_generator = train_datagen.flow_from_directory(
    'imagesprep/train',
    batch_size=32,
    class_mode='binary',
    target_size=(224,224))

validation_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input)

##brauch dann ein update auf das val set!!
validation_generator = validation_datagen.flow_from_directory(
    'imagesprep/train',
    shuffle=False,
    class_mode='binary',
    target_size=(224,224))

Found 277524 images belonging to 2 classes.
Found 277524 images belonging to 2 classes.


In [3]:
conv_base = ResNet50(
    include_top=False,
    weights='imagenet')

for layer in conv_base.layers:
    layer.trainable = False

Instructions for updating:
Colocations handled automatically by placer.


/opt/conda/envs/Python36/lib/python3.6/site-packages/keras_applications/resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


94658560/94653016 [==============================] - 1s 0us/step


## 3. Create a CNN (based on ResNet50)

In [4]:
x = conv_base.output
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dense(128, activation='relu')(x) 
predictions = layers.Dense(2, activation='softmax')(x)
model = Model(conv_base.input, predictions)

In [5]:
optimizer = keras.optimizers.Adam()
model.compile(loss='sparse_categorical_crossentropy',
              optimizer=optimizer,
              metrics=['accuracy'])

## 4. Train

In [ ]:
history = model.fit_generator(generator=train_generator,
                              #steps_per_epoch = 4, # nur für dry run da ja gar nich alles berücksichtigt wird
                              steps_per_epoch=ceil(1534/ 32),  
                              epochs=20,
                              validation_data=validation_generator,
                              validation_steps= 25,
                              callbacks = [EarlyStopping(monitor='val_loss', patience=3)]
                             )

Instructions for updating:
Use tf.cast instead.
Epoch 1/20
48/48 [==============================] - 301s 6s/step - loss: 0.4231 - acc: 0.8132 - val_loss: 0.0050 - val_acc: 1.0000
Epoch 2/20
48/48 [==============================] - 289s 6s/step - loss: 0.3583 - acc: 0.8522 - val_loss: 0.0181 - val_acc: 1.0000
Epoch 3/20
48/48 [==============================] - 290s 6s/step - loss: 0.3271 - acc: 0.8542 - val_loss: 0.0021 - val_acc: 1.0000
Epoch 4/20
48/48 [==============================] - 289s 6s/step - loss: 0.3362 - acc: 0.8483 - val_loss: 0.0035 - val_acc: 1.0000
Epoch 5/20
48/48 [==============================] - 296s 6s/step - loss: 0.3445 - acc: 0.8522 - val_loss: 0.0016 - val_acc: 1.0000
Epoch 6/20
48/48 [==============================] - 297s 6s/step - loss: 0.3449 - acc: 0.8444 - val_loss: 0.0034 - val_acc: 1.0000
Epoch 7/20
48/48 [==============================] - 294s 6s/step - loss: 0.3160 - acc: 0.8639 - val_loss: 6.8975e-04 - val_acc: 1.0000
Epoch 8/20
48/48 [=============

In [ ]:
print(history.history.keys())

In [ ]:
plt.figure(1, figsize = (15,8)) 
    
plt.subplot(221)  
plt.plot(history.history['acc'])  
plt.plot(history.history['val_acc'])  
plt.title('model accuracy')  
plt.ylabel('accuracy')  
plt.xlabel('epoch')  
plt.legend(['train', 'valid']) 
    
plt.subplot(222)  
plt.plot(history.history['loss'])  
plt.plot(history.history['val_loss'])  
plt.title('model loss')  
plt.ylabel('loss')  
plt.xlabel('epoch')  
plt.legend(['train', 'valid']) 

plt.show()